In [60]:
import torch
import pandas as pd
import os

In [61]:
path='.\\io_test\\fair_cut.csv'
shards_path='.\\io_test\\shards'

In [62]:
head=pd.read_csv(path, nrows=10)
cols=head.columns.to_list()
print(len(cols))

80


In [63]:
#DISSECT DATASET
def dissect(path_of_jumbo_ds, mem_limit, where_to_store):
    count=0
    with pd.read_csv(path, chunksize=mem_limit) as reader:
        for c in reader:
            c.to_csv(where_to_store + '\\shard_{0}.csv'.format(count), mode='w', header=True, index=False)
            count+=1

In [65]:
dissect(path, 1_000_000, shards_path)

In [99]:
from torch.utils.data import Dataset, DataLoader

class CustomDatasetShard(Dataset):
    def __init__(self, shard_path, features_cols, target, shard_length):
        self.x_cols, self.y_cols = features_cols, target
        self.path=shard_path
        self.len = shard_length

    def __getitem__(self, index):
        row = pd.read_csv(self.path,
                          skiprows=index-1,
                          nrows=1)
        return torch.tensor(row.iloc[0, self.x_cols], dtype=torch.float32), row.iloc[0, self.y_cols][0]
        
    def __len__(self):
        return self.len

In [101]:
shards = []
sh_len = [1_000_000,1_000_000,1_000_000,1_000_000,1_000_000,1_000_000,1_000_000,1_000_000,1_000_000,1_000_000,1_000_000, 548_461]
for dirname, _, filenames in os.walk(shards_path):
    for filename in filenames:
        shard = os.path.join(dirname, filename)
        shards.append(shard)


dss = []
for sh, l in zip(shards, sh_len):
    dss.append(CustomDatasetShard(sh, list(range(0,5)), [79], l))#list(range(0,79))

chain_ds=torch.utils.data.ConcatDataset(dss)
loader=DataLoader(chain_ds, shuffle=False, batch_size=10)

In [103]:
for features, targets in loader:
    print(features.shape, targets)

torch.Size([10, 5]) ('Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign')
torch.Size([10, 5]) ('Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign')
torch.Size([10, 5]) ('Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign')
torch.Size([10, 5]) ('Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign')
torch.Size([10, 5]) ('Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign')
torch.Size([10, 5]) ('Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign')
torch.Size([10, 5]) ('Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign')
torch.Size([10, 5]) ('Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign')
torch.Size([10, 5]) ('Benign', '

KeyboardInterrupt: 